In [2]:
#load libraries
import os, sys, math, glob
import pandas as pd
import numpy as np

In [3]:
def readin(scr_path):
# First argument is scr .txt file input
    input_scr = glob.glob(scr_path)
    input_scr = sorted([i for i in input_scr if 'era' not in i])
    subids = [os.path.basename(s).split('/')[-1][:-4] for s in input_scr]
    return dict(zip(input_scr, subids))

In [25]:
scrs = readin('/Users/ramihamati/Documents/PhD_Work/AVL/SCR/SCR_JUNE2021/**/AVLRAC-301-FC.txt')

In [28]:

for item, value in scrs.items():
    scr = pd.read_table(item, header=0, names=['microsiemens', 'stst', 'csps', 'csp', 'csm'], delim_whitespace=True)
    scr = scr.assign(time=[0 + (0.005)*i for i in range(len(scr))])[['time'] + scr.columns.tolist()]

    pct_change_columns = ['stst', 'csps', 'csp', 'csm']
    markers = []

    column_thresholds = {
        'stst': 8,
        'csps': 24,
        'csp': 56,
        'csm': 56
    }

    for col in pct_change_columns:
        scr[f'{col}_pct_change'] = scr[col].pct_change()
        
        if col == 'csm':
            potential_indices = scr.index[(scr[col] == 5) & (scr[f'{col}_pct_change'] > 0)]
            if len(potential_indices) == 60:
                faulty_indices = [0, 15, 30, 45]
                scr.loc[potential_indices[faulty_indices], col] = 0  # Set faulty indices to 0
                print(f"\nAutomatically removed faulty CSM indices: {[potential_indices[i] for i in faulty_indices]} for {value}")
        
        indices = scr.index[(scr[col] == 5) & (scr[f'{col}_pct_change'] > 0)]
        
        if len(indices) > column_thresholds[col]:
            print(f"\nColumn '{col}' has {len(indices)} markers (threshold: {column_thresholds[col]}):")
            for idx in indices:
                print(f"Index: {idx}, Time: {scr.loc[idx, 'time']}")
            
            to_delete = input(f"Enter indices to delete for '{col}' (comma-separated), or press Enter to keep all for {value}: ")
            if to_delete:
                delete_indices = [int(i.strip()) for i in to_delete.split(',')]
                scr.loc[delete_indices, col] = 0  # Set user-specified indices to 0
                indices = scr.index[(scr[col] == 5) & (scr[f'{col}_pct_change'] > 0)]  # Recalculate indices
        
        markers.extend([[scr.loc[idx, 'time'], col] for idx in indices])

    newpar = pd.DataFrame(markers, columns=['time','nid'])
    newpar['nid'] = newpar['nid'].replace({'stst':0, 'csps':3, 'csp':1, 'csm':2})
    newpar['time'] = newpar['time'].round(4)
    newpar.sort_values(by='time', inplace=True, ignore_index=True)
    newpar.to_csv(f'/Users/ramihamati/Documents/PhD_Work/AVL/SCR/syncedpars/{value}.synced.new.par', index=False, header = True, sep='\t')
    print(f'saved timestamps to: /Users/ramihamati/Documents/PhD_Work/AVL/SCR/syncedpars/{value}.synced.new.par')



Column 'stst' has 11 markers (threshold: 8):
Index: 101325, Time: 506.625
Index: 101770, Time: 508.85
Index: 574056, Time: 2870.28
Index: 602061, Time: 3010.305
Index: 703957, Time: 3519.785
Index: 708416, Time: 3542.08
Index: 822971, Time: 4114.8550000000005
Index: 829696, Time: 4148.4800000000005
Index: 928138, Time: 4640.6900000000005
Index: 931554, Time: 4657.77
Index: 1029975, Time: 5149.875

Column 'csps' has 27 markers (threshold: 24):
Index: 101221, Time: 506.105
Index: 101666, Time: 508.33
Index: 574056, Time: 2870.28
Index: 612154, Time: 3060.77
Index: 623956, Time: 3119.78
Index: 638157, Time: 3190.785
Index: 650358, Time: 3251.79
Index: 654957, Time: 3274.785
Index: 688358, Time: 3441.79
Index: 727569, Time: 3637.8450000000003
Index: 756771, Time: 3783.855
Index: 760571, Time: 3802.855
Index: 776773, Time: 3883.8650000000002
Index: 787173, Time: 3935.8650000000002
Index: 796373, Time: 3981.8650000000002
Index: 832735, Time: 4163.675
Index: 858536, Time: 4292.68
Index: 8807